In [1]:
import pandas as pd
import numpy as np
import configparser
import pymysql
import requests
import re
from tqdm import tqdm
import configparser
import openai
import os

In [2]:
config=configparser.ConfigParser()
config.read('/home/ubuntu/infoedge/llama-recipes/examples/configs/config.ini')

['/home/ubuntu/infoedge/llama-recipes/examples/configs/config.ini']

In [5]:
# openai.api_key=config['key']['infoedge']



In [6]:
# os.environ['OPENAI_API_KEY'] = openai.api_key

In [7]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.document_loaders import TextLoader
# from langchain.document_loaders import Py

from langchain.text_splitter import RecursiveCharacterTextSplitter 

In [8]:
# loader = PyPDFDirectoryLoader('/home/ubuntu/infoedge/llama-recipes/examples/custom_data/user_resume_data/')

In [27]:
model = 'meta-llama/Llama-2-7b-chat-hf'

In [10]:
DATA_PATH = '/home/ubuntu/infoedge/llama-recipes/examples/custom_data/user_resume_data/uid_1103243.txt' #Your root data folder path
DB_FAISS_PATH = 'vectorstore/db_faiss'

In [11]:
loader = TextLoader(DATA_PATH)
documents = loader.load()

In [18]:
# documents

In [13]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=50, chunk_overlap=10)
splits = text_splitter.split_documents(documents)
print(len(splits), splits[0])

147 page_content='User ID: 1103243' metadata={'source': '/home/ubuntu/infoedge/llama-recipes/examples/custom_data/user_resume_data/uid_1103243.txt'}


In [19]:
print(len(documents), documents[0].page_content[0:100])


1 User ID: 1103243
ASHWATH PARTHASARATHY Phone: 0-9884178474, Residence: 0-8939374135 ~E-Mail: ashwath


In [20]:
embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2',
                                       model_kwargs={'device': 'cpu'})

In [26]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={'device': 'cpu'}, encode_kwargs={}, multi_process=False)

In [21]:
db = FAISS.from_documents(splits, embeddings)
db.save_local(DB_FAISS_PATH)

In [22]:
# !curl localhost:5001/generate -X POST -H 'Content-Type: application/json' -d '{"inputs": "What is good about Beijing?", "parameters": { "max_new_tokens":64}}' #Replace the locahost with the IP visible to the machine running the notebook     


In [28]:
import langchain
from queue import Queue
from typing import Any
from langchain.llms.huggingface_text_gen_inference import HuggingFaceTextGenInference
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.schema import LLMResult
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.prompts.prompt import PromptTemplate
from anyio.from_thread import start_blocking_portal #For model callback streaming

langchain.debug=True 

#vector db path
DB_FAISS_PATH = 'vectorstore/db_faiss'

#Llama2 TGI models host port
LLAMA2_7B_HOSTPORT = "http://localhost:8080/" #Replace the locahost with the IP visible to the machine running the notebook
LLAMA2_13B_HOSTPORT = "http://localhost:8080/" #Add your own host ports for model switching. You can host another TGI model on same instance on a different port.


model_dict = {
    # "7b-chat" : LLAMA2_7B_HOSTPORT,
    # "13b-chat" : LLAMA2_13B_HOSTPORT,
    "7b-chat" : 'meta-llama/Llama-2-7b-chat-hf'
}

system_message = {"role": "system", "content": "You are a helpful assistant."}

In [30]:
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2",
                                       model_kwargs={'device': 'cpu'})
db = FAISS.load_local(DB_FAISS_PATH, embeddings)


In [49]:
y = "https://api-inference.huggingface.co/models/meta-llama/Llama-2-7b-chat-hf"

In [50]:
llm = HuggingFaceTextGenInference(
    inference_server_url=y,
    max_new_tokens=512,
    top_k=10,
    top_p=0.9,
    typical_p=0.95,
    temperature=0.6,
    repetition_penalty=1,
    do_sample=True,
    streaming=True
)

In [51]:
template = """
[INST]Use the following pieces of context to answer the question. If no context provided, answer like a AI assistant.
{context}
Question: {question} [/INST]
"""

retriever = db.as_retriever(
        search_kwargs={"k": 6}
    )

In [52]:
qa_chain = RetrievalQA.from_chain_type(
    llm=llm, 
    retriever=retriever,     
    chain_type_kwargs={
        "prompt": PromptTemplate(
            template=template,
            input_variables=["context", "question"],
        ),
    }
)

In [53]:
result = qa_chain({"query": "What are relationship manager career objectives?"})
print(result)

[chain/start] [1:chain:RetrievalQA] Entering Chain run with input:
{
  "query": "What are relationship manager career objectives?"
}
[chain/start] [1:chain:RetrievalQA > 3:chain:StuffDocumentsChain] Entering Chain run with input:
[inputs]
[chain/start] [1:chain:RetrievalQA > 3:chain:StuffDocumentsChain > 4:chain:LLMChain] Entering Chain run with input:
{
  "question": "What are relationship manager career objectives?",
  "context": "Relationship Management. Career Objective To take\n\npursue an ambitious career in an organisation\n\nthe college in 17 National Level Management Meets\n\nplan and ensuring that sales goals are met in\n\nbusiness goals and achievement of topline targets\n\nbeing an ambassador of our organisation. Assists"
}
[llm/start] [1:chain:RetrievalQA > 3:chain:StuffDocumentsChain > 4:chain:LLMChain > 5:llm:HuggingFaceTextGenInference] Entering LLM run with input:
{
  "prompts": [
    "[INST]Use the following pieces of context to answer the question. If no context prov

BadRequestError: Model requires a Pro subscription; check out hf.co/pricing to learn more. Make sure to include your HF token in your query.

In [63]:
from transformers import AutoModelForCausalLM, AutoTokenizer

# Define your model and tokenizer
# model_name = "EleutherAI/gpt-neo-1.3B"
# tokenizer = AutoTokenizer.from_pretrained(model_name)
# model = AutoModelForCausalLM.from_pretrained(model_name)


model = AutoModelForCausalLM.from_pretrained("microsoft/phi-2", torch_dtype="auto", trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained("microsoft/phi-2", trust_remote_code=True)

# Vector store and embeddings (assuming you want to use them)
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2", model_kwargs={'device': 'cpu'})
db = FAISS.load_local(DB_FAISS_PATH, embeddings)

# Function to generate text
def generate_text(prompt, max_length=50, num_return_sequences=1):
    inputs = tokenizer(prompt, return_tensors="pt")
    outputs = model.generate(**inputs, max_length=max_length, num_return_sequences=num_return_sequences, no_repeat_ngram_size=2)
    return tokenizer.batch_decode(outputs, skip_special_tokens=True)

# Example usage
prompt = "What are relationship manager career objectives"
generated_text = generate_text(prompt)
print("Generated Text:", generated_text)


config.json:   0%|          | 0.00/866 [00:00<?, ?B/s]

configuration_phi.py:   0%|          | 0.00/9.26k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/phi-2:
- configuration_phi.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_phi.py:   0%|          | 0.00/62.5k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/phi-2:
- modeling_phi.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


model.safetensors.index.json:   0%|          | 0.00/35.7k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

KeyboardInterrupt: 

## Direct RAG example

In [56]:
openai.api_key=config['key']['infoedge']



In [5]:
os.environ['OPENAI_API_KEY'] = openai.api_key

In [6]:
from langchain.indexes import VectorstoreIndexCreator
from langchain.document_loaders import TextLoader


loader = TextLoader("/home/ubuntu/infoedge/llama-recipes/examples/custom_data/user_resume_data/uid_1103243.txt")
index = VectorstoreIndexCreator().from_loaders([loader])
index.query("What are relationship manager career objectives")

' The career objective of a relationship manager is to pursue an ambitious career in an organization where there are ample learning opportunities, upward mobility, exposure to innovation, and motivation. They aim to exhibit their resourcefulness in both technical and organizational skills and to take on challenging roles in key account management, travel and expense management solutions, and relationship management.'